In [6]:
from world.sim import Sim
import pickle
import networkx as nx
from decision_making.gen_models import query_priors_gen
import decision_making.planner as p
from decision_making.bayes_models import ObjBayesNet
from decision_making.util_classes import AgentState, PlannerParams
from decision_making.util_functions import combined, a_star, obs_env_prep
import random as rd
import threading



#### Takes the path of a pickle file containing a world and unwraps it for the planner

In [7]:
def load_world_for_animation(file_path:str):
    """
    Given a file-path string, loads a grid-world, its underlying scene-graph, and the graph attributes.
    """
    stored_dict = pickle.load(open(file_path, 'rb'))
    world = stored_dict["grid world"] # 2D numpy array
    sg = nx.Graph(stored_dict["scene graph"]) # Full 3D scene-graph
    attr_dict = stored_dict["node attrs"] # node attributes
    nx.set_node_attributes(sg, attr_dict) # set node attributes TODO: figure out if we can save SGs with node attributes in pickle file
    return world, sg, attr_dict

#### Set up the planner and animation-pane

In [8]:
def run_animation(sg, world, obj):
    """
    Runs a single animation for a specific environment and target object
    """
    start_point, starting_graph = obs_env_prep(sg, po=True) 
    # start_point = "living_room0"
    init_state = AgentState(start_point, sg.nodes[start_point]["centroid"])
    root = "house"
    # print([node for node in sg.neighbors("laptop0")])
    ML_TEST = {"cost_scaler": [0, 1, 0], "horizon_len": 1, "traj_samples": 500}
    LAZY_TEST = {"cost_scaler": [0, 0, 1], "horizon_len": 5, "traj_samples": 500}
    MYOPIC = {"cost_scaler": [0.001, 100, 1], "horizon_len": 1, "traj_samples": 500}
    MINE = {"cost_scaler": [0, 1, 1], "horizon_len": 3, "traj_samples": 50}
    to_test = MINE


    # TODO move this somewhere else
    # if world is not None:
    point = sg.nodes[start_point]["centroid"]
    neighbors = [(-1, 0), (0, -1), (1, 0), (0, 1)]
    while world[point] != 0: # check if floor
        adj = rd.choice(neighbors)
        next = (point[0] + adj[0], point[1] + adj[1])
        if world[next] == 1: # check if wall; if so, don't even bother
            continue
        point = next

    init_state = AgentState(start_point, point)
    print("Starting point: " + str(point))

    params = PlannerParams(init_state, starting_graph, obj, root, to_test["cost_scaler"], to_test["horizon_len"], 500, combined, {0, 1})
    bn = ObjBayesNet
    print(f"Starting in {start_point}, finding {obj}")

    sim_obs = Sim(sg, world)
    plan = p.Planner(params, query_priors_gen, bn, lambda obs:True if obs == 1 else False, 
                        external_controller=sim_obs.decision_outcome, distance_func=a_star, world=world, animation=sim_obs,
                        greedy_policy=False)

    # Start the planning thread
    continue_planning = threading.Event()

    path_thread = threading.Thread(target=plan.planner, args=(continue_planning, ))
    path_thread.start()

    animation_thread = threading.Thread(target=sim_obs.animate_agent, args=(init_state.position, continue_planning))
    animation_thread.start()

In [9]:
def animate(object, demo):
    world, sg, _ = load_world_for_animation(f'world/demo_houses/{demo}.pickle')
    run_animation(sg, world, object)

#### Run the animation! The agent's actions are printed parallel to the simulation

In [10]:
animate(object="laptop", demo="demo1")

Starting point: (8, 14)
Starting in living_room0, finding laptop


(8, 14)

hi:  {'couch0': 0.23546890264434073, 'tv_set0': 0.1282698335851483, 'bookshelf0': 0.15080953547103013, 'chair0': 0.11424128573751913, 'utility_closet0': 0.12141491255423725, 'bedroom0': 0.24979553000772445}

I chose the couch0
Planning path...

hi:  {'tv_set0': 0.16777582236851415, 'bookshelf0': 0.1972575556346188, 'chair0': 0.1494266042711068, 'utility_closet0': 0.1588096455123723, 'bedroom0': 0.32673037221338785}

I chose the bookshelf0
Planning path...

hi:  {'tv_set0': 0.20900330304715803, 'chair0': 0.18614513947775368, 'utility_closet0': 0.19783387140805966, 'bedroom0': 0.40701768606702854}

I chose the tv_set0
Planning path...

hi:  {'chair0': 0.2353298568689869, 'utility_closet0': 0.25010707651520603, 'bedroom0': 0.5145630666158071}

I chose the bedroom0
Planning path...

hi:  {'chair0': 0.07130864084395623, 'bed0': 0.15529272035490052, 'desk0': 0.5441199800925652, 'nightstand0': 0.0636772436847392, 'utility_closet0': 0.07578637036984098, 'basement0': 0.0898150446539976